In [10]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [11]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [12]:
config = get_config(nb_name)

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [20]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [15]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [16]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [17]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [18]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     80    8  463  9167.40  1851  9145.84  21.31  0.28  8881.64  1726   
10000    80   17  360  9126.04  1788  9103.30  22.52  0.23  8870.14  1709   
15000    78   26  257  9108.23  1762  9085.20  22.81  0.21  8867.22  1704   
20000    77   35  154  9098.05  1748  9074.91  22.91  0.20  8864.86  1701   
25000    75   44   51  9091.08  1738  9068.03  22.95  0.19  8863.46  1699   
...     ...  ...  ...      ...   ...      ...    ...   ...      ...   ...   
325000   78  573  108  9051.54  1683  9030.67  22.01  0.12  8862.93  1697   
330000   78  582    5  9051.37  1683  9030.55  22.00  0.12  8863.21  1698   
335000   78  590  469  9051.24  1682  9030.44  21.99  0.12  8862.44  1696   
340000   78  599  366  9051.12  1682  9030.35  21.98  0.12  8862.90  1697   
345000   77  608  263  9051.00  1682  9030.25  21.97  0.12  8863.67  1697   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    8857.70  23.73  0.21  8881.08  1726  0.41  0.45  0.46  0.31  0.22  
10000   8846.55  23.42  0.18  8870.38  1709  0.44  0.46  0.48  0.30  0.21  
15000   8843.71  23.34  0.17  8866.46  1703  0.45  0.46  0.49  0.30  0.20  
20000   8841.53  23.16  0.16  8865.50  1702  0.47  0.47  0.49  0.28  0.20  
25000   8840.34  22.95  0.16  8862.96  1698  0.51  0.47  0.49  0.28  0.20  
...         ...    ...   ...      ...   ...   ...   ...   ...   ...   ...  
325000  8841.59  21.25  0.09  8861.52  1694  0.54  0.48  0.49  0.28  0.20  
330000  8841.97  21.15  0.09  8861.52  1694  0.54  0.48  0.49  0.28  0.20  
335000  8841.24  21.11  0.09  8861.52  1694  0.53  0.48  0.48  0.28  0.20  
340000  8841.64  21.17  0.09  8861.52  1694  0.53  0.48  0.48  0.28  0.20  
345000  8842.49  21.09  0.09  8861.52  1694  0.53  0.48  0.49  0.28  0.20  

[69 rows x 20 columns]

0 R: 1.000 P: 0.065 user dialogue knowledge utterance generation domain action state systems human
   1 R: 0.184 P: 0.045 models embeddings neural vector network training vectors representations embedding representation
     11 R: 0.076 P: 0.076 document documents question topic query sentences questions answer method similarity
     12 R: 0.063 P: 0.063 sentiment features tweets negative positive classification analysis social opinion polarity
   2 R: 0.212 P: 0.065 tree parsing dependency parser grammar node trees parse rules nodes
     21 R: 0.044 P: 0.044 event relations annotation relation discourse events argument semantic annotated temporal
     22 R: 0.103 P: 0.103 verb semantic syntactic structure noun lexical verbs rules grammar phrase
   3 R: 0.208 P: 0.079 training models algorithm probability features feature function learning performance distribution
     31 R: 0.077 P: 0.077 features feature training performance entity learning classification classifier entities class
  

KeyboardInterrupt: 

# analysis

In [34]:
probs_topics = []
years = []
for i, train_batch in train_batches:
    probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(train_batch, mode='test'))
    years_batch = [instance.year for instance in train_batch]
    probs_topics += [probs_topics_batch]
    years += years_batch
probs_topics = np.concatenate(probs_topics)
years = np.array(years)

In [55]:
topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
probs_topics.shape, years.shape

((36288, 16), (36288,))

In [59]:
topic_year

{0: 2006.1405287293783,
 1: 2015.073065684627,
 2: 2006.6358226095088,
 3: 2010.2605308626469,
 4: 2011.1153424999409,
 5: 2010.2201559271937,
 11: 2010.638231762466,
 12: 2013.6412160983643,
 21: 2009.3432210176409,
 22: 2003.0377479786164,
 31: 2010.7845425693845,
 32: 2007.4959372377405,
 41: 2008.9745451250799,
 42: 2009.024079592822,
 51: 2007.963976828227,
 52: 2008.7701026307714}

In [64]:
def print_topic_sample(sess, model, topic_prob_topic=None, recur_prob_topic=None, topic_freq_tokens=None, topic_year=None, parent_idx=0, depth=0):
    if depth == 0: # print root
        assert len(topic_prob_topic) == len(recur_prob_topic) == len(topic_freq_tokens) == len(topic_year)
        freq_tokens = topic_freq_tokens[parent_idx]
        recur_topic = recur_prob_topic[parent_idx]
        prob_topic = topic_prob_topic[parent_idx]
        year = topic_year[parent_idx]
        print(parent_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
    
    child_idxs = model.tree_idxs[parent_idx]
    depth += 1
    for child_idx in child_idxs:
        freq_tokens = topic_freq_tokens[child_idx]
        recur_topic = recur_prob_topic[child_idx]
        prob_topic = topic_prob_topic[child_idx]
        year = topic_year[child_idx]
        print('  '*depth, child_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
        
        if child_idx in model.tree_idxs: 
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year, parent_idx=child_idx, depth=depth)

In [65]:
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2006 user dialogue knowledge utterance generation domain action state systems human
   1 Avg Year: 2015 models embeddings neural vector network training vectors representations embedding representation
     11 Avg Year: 2010 document documents question topic query sentences questions answer method similarity
     12 Avg Year: 2013 sentiment features tweets negative positive classification analysis social opinion polarity
   2 Avg Year: 2006 tree parsing dependency parser grammar node trees parse rules nodes
     21 Avg Year: 2009 event relations annotation relation discourse events argument semantic annotated temporal
     22 Avg Year: 2003 verb semantic syntactic structure noun lexical verbs rules grammar phrase
   3 Avg Year: 2010 training models algorithm probability features feature function learning performance distribution
     31 Avg Year: 2010 features feature training performance entity learning classification classifier entities class
     32 Avg Year: 2007 errors